In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam

In [2]:
train_datagen = ImageDataGenerator(rescale=1.0/255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1.0/255)

train_data = train_datagen.flow_from_directory('train_df',
                                              target_size=(128, 128),
                                              batch_size=32,
                                              class_mode='binary')

test_data = test_datagen.flow_from_directory('test_df',
                                            target_size=(128, 128),
                                            batch_size=32,
                                            class_mode='binary')

Found 2562 images belonging to 2 classes.
Found 550 images belonging to 2 classes.


In [3]:
model = Sequential()

model.add(Conv2D(32, (3, 3), input_shape=(128, 128, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())

model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(1, activation='sigmoid'))

In [4]:
model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [5]:
model.fit(train_data,
          steps_per_epoch=train_data.samples // 32,
          epochs=3,
          validation_data=test_data,
          validation_steps=test_data.samples // 32)

Epoch 1/3
80/80 [==============================] - 45s 551ms/step - loss: 0.6223 - accuracy: 0.6597 - val_loss: 0.5917 - val_accuracy: 0.5643
Epoch 2/3
80/80 [==============================] - 40s 497ms/step - loss: 0.4073 - accuracy: 0.8431 - val_loss: 0.6442 - val_accuracy: 0.6801
Epoch 3/3
80/80 [==============================] - 35s 436ms/step - loss: 0.3761 - accuracy: 0.8593 - val_loss: 0.5268 - val_accuracy: 0.7647


In [6]:
loss, accuracy = model.evaluate(test_data)
print(f'Test loss: {loss:.4f}')
print(f'Test accuracy: {accuracy:.4f}')

18/18 [==============================] - 2s 94ms/step - loss: 0.5278 - accuracy: 0.7618
Test loss: 0.5278
Test accuracy: 0.7618


In [7]:
model.save('DR.h5')

C:\Users\gsuma\AppData\Roaming\Python\Python311\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [8]:
#classifying the image
from tensorflow.keras.models import load_model
model = load_model('DR.h5')
def preprocess_image(image_path):
    from tensorflow.keras.preprocessing import image
    img = image.load_img(image_path, target_size=(128, 128))
    img = image.img_to_array(img)
    img = img / 255.0 
    img = np.expand_dims(img, axis=0) 
    return img
def predict_image(image_path):
    img = preprocess_image(image_path)
    prediction = model.predict(img)

    if prediction > 0.5:
        return 'DR'
    else:
        return 'NON_DR'

image_path = r'test_df\DR\0ac436400db4.png'
result = predict_image(image_path)
print(f'The image is a {result}')
image_path


1/1 [==============================] - 0s 139ms/step
The image is a DR


'test_df\\DR\\0ac436400db4.png'

In [9]:
import os
from sklearn.metrics import *
test_dir = r'test_df'

true_labels = []
predicted_labels = []
for class_name in ['DR', 'No_DR']:
    class_dir = os.path.join(test_dir, class_name)
    for image_filename in os.listdir(class_dir):
        image_path = os.path.join(class_dir, image_filename)
        true_labels.append(class_name)

        result = predict_image(image_path)
        predicted_labels.append(result)

confusion = confusion_matrix(true_labels, predicted_labels)
accuracy = accuracy_score(true_labels, predicted_labels)
class_report = classification_report(true_labels, predicted_labels)


1/1 [==============================] - 0s 25ms/step


C:\Users\gsuma\AppData\Roaming\Python\Python311\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\gsuma\AppData\Roaming\Python\Python311\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\gsuma\AppData\Roaming\Python\Python311\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Use

In [10]:
print("Confusion Matrix:\n", confusion)
print("\nAccuracy:", accuracy)
print("\nClassification Report:\n", class_report)

Confusion Matrix:
 [[110 169   0]
 [  0   0   0]
 [250  21   0]]

Accuracy: 0.2

Classification Report:
               precision    recall  f1-score   support

          DR       0.31      0.39      0.34       279
      NON_DR       0.00      0.00      0.00         0
       No_DR       0.00      0.00      0.00       271

    accuracy                           0.20       550
   macro avg       0.10      0.13      0.11       550
weighted avg       0.15      0.20      0.17       550

